In [ ]:
# Authenticate google account

from google.colab import auth
auth.authenticate_user()

In [ ]:
# Install OpenAI
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import openai
import re
import os
from google.cloud import bigquery

openai_api_key = "<your-api-key>"
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
def text2query(prompt):

  # Setup: Priming the LLM to understand the task
  messages = []
  content = "For all responses after this message, you will generate queries from the following table: bigquery-public-data.idc_current.dicom_all. Also make sure to use regex when necessary. Please be as specific as possible and only return the final query enclosed in ```. Do not provide explanations."
  messages.append({"role": "user", "content": content})
  completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0301",
        messages=messages
  )
  chat_response = completion.choices[0].message.content
  messages.append({"role": "assistant", "content": chat_response})

  # Again prime the LLM + get response for your task

  messages.append({"role": "user", "content": prompt})
  completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-0301",
      messages=messages
  )
  chat_response = completion.choices[0].message.content
  messages.append({"role": "assistant", "content": chat_response})
  print(chat_response)

  # Parse the chat response and get the query
  query = parse(chat_response)
  print (query)

  # run the big query and get the response dataframe 
  df,e,success = runBigQuery(query[0])
  k = 1
  while success==0 and k < 10:
    messages.append({"role": "user", "content": str(e)})
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-0301",
      messages=messages
    )
    chat_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": chat_response})
    print(chat_response)
    query = parse(chat_response)
    print (query)
    df,e,success = runBigQuery(query[0])
    k += 1
  
  print(df)

def parse(chat_response):
  code_blocks = re.findall(r"```(.+?)```", chat_response, re.DOTALL)
  return code_blocks
  

def runBigQuery(query):
  # Construct a BigQuery client object.
  e = 0
  df = 0
  success = 0
  try:
    client = bigquery.Client(project='chexviz')
    df = client.query(query).to_dataframe()
    print(df.head())
    success = 1
  except Exception as e:
    print(str(e))
    success = 0
    return df,e, success
  return df,e,success

In [ ]:
text2query('Select all unique lesions as deduced from SeriesDescription which is LIDC specific and the nodules are clustered by pylidc')

```SELECT DISTINCT SeriesDescription FROM `bigquery-public-data.idc_current.dicom_all` WHERE SeriesDescription LIKE '%LIDC-IDRI-#%%#-#%%%'```
["SELECT DISTINCT SeriesDescription FROM `bigquery-public-data.idc_current.dicom_all` WHERE SeriesDescription LIKE '%LIDC-IDRI-#%%#-#%%%'"]
Empty DataFrame
Columns: [SeriesDescription]
Index: []
Empty DataFrame
Columns: [SeriesDescription]
Index: []
